In [ ]:
import xarray as xr

In [ ]:
# The air-temperature dataset we looked at previously is a Dask array
# together with coordinate and other meta-data variables.
ds = xr.tutorial.open_dataset('air_temperature',
                              chunks={'lat': 25, 'lon': 25, 'time': -1})
display(ds.air.data)
print(ds.air.data)

In [ ]:
# We can look at how to evaluation works in this case
ds.air.data.max().visualize()

In [ ]:
# we can extend to chunk multiple files
!git clone https://github.com/pangeo-data/tutorial-data.git 

In [ ]:
dsnc=xr.open_mfdataset('tutorial-data/sst/*.nc')

In [ ]:
import numpy as np
np.nanmax(dsnc.sst.data).visualize()

In [ ]:
import dask
import graphviz
from dask.dot import dot_graph, task_label, label, to_graphviz
?to_graphviz

In [ ]:
import xarray as xr
mur_sst = xr.open_zarr('https://mur-sst.s3.us-west-2.amazonaws.com/zarr-v1',consolidated=True)

In [ ]:
import xarray as xr
mur_sst = xr.open_zarr('https://mur-sst.s3.us-west-2.amazonaws.com/zarr-v1',consolidated=True)
display(mur_sst)
display(mur_sst.analysed_sst.data.blocks[0,0,0])

In [ ]:
mur_sst.analysed_sst.data.blocks[0,0,0]

In [ ]:
mur_sst.analysed_sst.data.blocks[0,0,0][0,0,0].compute()

In [ ]:
%time sst_ts_handle=mur_sst.analysed_sst.sel(lat=41.62,method='nearest').sel(lon=slice(-69.0,-68.0),time=slice('2002-06-01','2003-06-01'))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(sst_ts-273)

In [ ]:
sst_ts_handle

In [ ]:
ds=mur_sst.analysed_sst
mask_lon=(ds.lon >= -71.5) & ( ds.lon <= -68)
mask_lat=(ds.lat >=  41) & ( ds.lat <=  43)
import dask
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ds_masked=ds.where(mask_lon & mask_lat, drop=True)

In [ ]:
sst_patch=ds_masked.data.blocks[0,0,0][0,:,:].compute()

In [ ]:
ds_masked.data.blocks[0,0,0][0,:,:]

In [ ]:
%time sst_patch=ds_masked.data.blocks[0,0,0][0,:,:].compute()

In [ ]:
plt.contourf(sst_patch-273);plt.colorbar()

In [ ]:
%time sst_patches=ds_masked.data.blocks[0:10,0,0][0:10,:,:].compute()

In [ ]:
plt.contourf(sst_patches[5,:,:]-273)

In [ ]:
ds_masked.data.blocks[0:10,0,0][0:10,:,:]

In [ ]:
%time sst_patches=ds_masked.sel(time=slice('2002-06-01','2003-06-01')).compute()

In [ ]:
import numpy as np
np.nanmin(sst_patches)

In [ ]:
plt.contourf(sst_patches[0,:,:],vmax=mn+2*mv,vmin=mn-2*mv,cmap=cmap);plt.colorbar()

In [ ]:
[np.nanmean(sst_patches), np.nanstd(sst_patches)]

In [ ]:
mn=np.nanmean(sst_patches)
mv=np.nanstd(sst_patches)

In [ ]:
cmap = plt.cm.get_cmap("gist_ncar")

In [ ]:
sst_patches[0,:,:]